# Lab01: Deep Neural Networks in pytorch

## 배울 내용

- Pytorch를 이용한 간단한 neural network 작성방법
- torch, torch.nn, torch.utils.data와 같은 submodule이 학습과 추론 과정을 효율적으로 작성하게 해주는지

핵심적으로 Pytorch 라이브러리는 아래의 역할을 수행해준다.    
- 행렬 계산
- 자동 미분
- GPU 가속과 여러 노드 분배

대부분의 시간에 우리는 Pytorch의 핵심 기능을 제거하는 일을 해본다. ([What is `torch.nn` really?](https://pytorch.org/tutorials/beginner/nn_tutorial.html). 참고)     
우리는 ML와 DNNs 구조에 대해 익숙하다는 것을 전제에 두기 때문에 refreshing을 위해서 아래의 영상을 참고하는 것을 추천한다.      
[3Blue1Brown's videos](https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&ab_channel=3Blue1Brown)
or
[the NYU course on deep learning by Le Cun and Canziani](https://cds.nyu.edu/deep-learning/)

# Setup

Google colab을 사용한다면 아래의 cell은 필요한 모든 환경을 설치해준다.

In [1]:
lab_idx = 1

if "bootstrap" not in locals() or bootstrap.run:
    # path management for Python
    pythonpath, = !echo $PYTHONPATH
    if "." not in pythonpath.split(":"):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    # change into the lab directory
    bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow "hot-reloading" of modules
    %load_ext autoreload
    %autoreload 2

    bootstrap.run = False  # change to True re-run setup
    
!pwd
%ls

env: PYTHONPATH=.:/env/python
.:/env/python
/content/fsdl-text-recognizer-2022-labs/lab01
notebooks/  text_recognizer/


# Getting data and making `Tensor`s

모델을 만들기 전에 우리는 데이터가 필요하다.     
아래의 코드는 [MNIST dataset of handwritten digits](https://en.wikipedia.org/wiki/MNIST_database)를 다운받기위해 파이썬 표준 라이브러리를 사용한다.     
       
       
최근 state-of-art 모델에 사용되는 데이터는 일반적으로 매우 크기 때문에 단일 모델에 저장되지 않는다(RAM은 말할 것도 없이)      
따라서 네트워크를 통해 데이터를 가져오는 것은 모델을 훈련시키기 위해 일반적으로 수행하는 첫 번째 일이다.

In [2]:
from pathlib import Path
import requests


def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename


data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

더 큰 데이터는 더 많은 자원을 사용한다. -- 네트워크를 통해 데이터를 읽고 쓰고 보낼때 -- 따라서 데이터 셋은 압축되어 있다.(`.gz` 확장자)      
각 데이터셋 조각들(훈련과 검증데이터의 input과 output)은 단일 파이썬 객체이다.(array)     
우리는 `pickle` 라이브러리를 이용하여(`.pkl` 확장자) 파이썬 객체를 디스크에 저장할 수 있고(serialization) 다시 불러올 수 있다(deserialization)

In [3]:
import gzip
import pickle


def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)

파이토치는 자체적으로 array를 표현하는 타입인 `torch.Tensor`를 가지고 있다. 아래의 cell은 array를 `torch.Tensor` 타입으로 바꿔준다.      
"ML" 에서의 "tensor"는 물리, 수학, 컴퓨터공학 등에서 이야기하는 tensor와 의미가 다르다.     
간단하게 말하면 "ML"에서의 tensor는 단지 2차원 이상의 array를 의미한다.

In [4]:
import torch


x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

텐서들은 그들이 포함하고 있는 array의 내용에 따라 작성된다.

In [5]:
print(x_train, y_train, sep="\n")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])


텐서에 접근하는 것은 "indexing"이라고 불린다.    기본적인 파이썬 인덱싱 syntax와 같이, 이는 `항상 새로운 Tensor를 반환`한다.

In [6]:
y_train[0], x_train[0, ::2]

(tensor(5),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.4922, 0.6836, 0.6484,
         0.9648, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1172, 0.3672,
         0.6641, 0.9883, 0.9883, 0.8789, 0.9883, 0.7617, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.9297, 0.9883, 0.9883, 0.9883, 0.9883, 0.3633,
         0.3203,

대부분의 array 연산을 수행하는 고성능 라이브러리와 마찬가지로 PyTorch는 Tensor에 대한 metadata를 제공하는 방법이 존재한다.      
가장 중요한 metadata 중 하나는 dimension과 shape 이다.      
dimension은 우리가 array에서 숫자를 얻기 위해 필요한 index의 수를 지정한다.

In [7]:
x_train.ndim, y_train.ndim

(2, 1)

In [8]:
x_train[0, 0], y_train[0]

(tensor(0.), tensor(5))

`y_train`과 같은 1차원 텐서의 경우 shape는 얼마나 많은 항목을 가지고 있는지를 보여준다.     
`x_trian`과 같은 2차원 텐서에서 shape는 해당 텐서의 row와 column의 수를 나타내준다.

In [9]:
n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

torch.Size([50000, 784])
torch.Size([50000])


이 metadata는 파이썬의 다른 객체들의 metadata와 비슷한 역할을 수행한다.      
즉, 객체가 함수에 적절한 input과 output인지 정보를 알려주는 역할을 한다.     
대부분의 `Tensor`객체를 받는 함수들(indexing, 행렬 곱 등)은 특정 형태와 차원을 갖는 `Tensor`객체만을 받는다.     
따라서 계산 중간에 `ndim`과 `shape`를 사용하여 `Tensor`객체의 형태를 확인하는 것은 매우 중요한 디버깅 작업이다.

We won't spend much time here on writing raw array math code in PyTorch,
nor will we spend much time on how PyTorch works.

> If you'd like to get better at writing PyTorch code,
try out
[these "Tensor Puzzles" by Sasha Rush](https://github.com/srush/Tensor-Puzzles).
We wrote a bit about what these puzzles reveal about programming
with arrays [here](https://twitter.com/charles_irl/status/1517991568266776577?s=20&t=i9cZJer0RPI2lzPIiCF_kQ).

> If you'd like to get a better understanging of the internals
of PyTorch, check out
[this blog post by Edward Yang](http://blog.ezyang.com/2019/05/pytorch-internals/).

As we'll see below,
`torch.nn` provides most of what we need
for building deep learning models.

`x_train` 안에 담겨있는 텐서는 임의의 값이 아니라, 손으로 쓰여진 숫자 이미지에 대한 정보를 담고 있다.     
`y_train`은 각 숫자의 정답에 해당하는 텐서를 가지고 있다.

In [10]:
import random
import wandb
import text_recognizer.metadata.mnist as metadata

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])
wandb.Image(example.reshape(*metadata.DIMS)).image

tensor(9)


이미지를 받고 해당하는 숫자를 결과로 출력하는 모델을 간단한 torch 모델부터 시작해서 실제 사용하는 모델로 발전시킨다.

# `torch.Tensor`와 python만 이용하여 DNN 구축하기

## Defining the model

torch로 텐서를 만들기 위해서는 `값`과, `미분 값을 사용할지 여부(reguires_grad)`를 전달해야한다.

In [11]:
import math
import torch

weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [12]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

모델의 output을 확률 값으로 바꾸기 위해 `softmax`를 이용한다.     
numerically stablity를 위해서 softmax에 logarithm을 계산한다. [it is more natural to work with the logarithms of probabilities](https://youtu.be/LBemXHm_Ops?t=1071).

$log({exp(x_i) \over {\sum_j exp(x_j)}}) = x_i - log({{\sum_j exp(x_j)}})$

In [13]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis=1))[:, None] # None sum 으로 인해 없어지는 axis=1 차원 유지

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

일반적으로, 학습시 데이터를 작은 "batch" 단위로 나누게 된다.

In [14]:
bs = 64
xb = x_train[0:bs]
outs = model(xb)

print(outs[0], outs.shape)

tensor([-2.2369, -1.9443, -2.6531, -2.2017, -2.5600, -2.0436, -2.5377, -2.2186,
        -2.5019, -2.3808], grad_fn=<SelectBackward0>) torch.Size([64, 10])


In [15]:
print(xb.shape)
print(torch.log(torch.sum(torch.exp(xb), axis=1))[:, None].shape)
print(torch.log(torch.sum(torch.exp(xb), axis=1))[:].shape)
print(x_train.shape)
print(x_train[:].shape)
print(x_train[:, :, None].shape)
print(x_train[:, None, :].shape)
print(x_train[:, None].shape)

torch.Size([64, 784])
torch.Size([64, 1])
torch.Size([64])
torch.Size([50000, 784])
torch.Size([50000, 784])
torch.Size([50000, 784, 1])
torch.Size([50000, 1, 784])
torch.Size([50000, 1, 784])


## loss와 평가 지표(metrics) 정의

모델의 결과를 정답 레이블와 일치기키기 위해 각 샘플당 가장 높은 확률을 갖는 클래스를 정답 레이블과 비교한다.

In [16]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [17]:
yb = y_train[0:bs]

acc = accuracy(outs, yb)

print(acc)

tensor(0.0312)


모델이 random하게 초기화되어 있기 때문에 성능이 좋지 못하다.

acc 자체는 backward 계산을 못하기 때문에 모델을 학습시키는데 사용할 수 없다.

In [18]:
try:
    acc.backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


분류 문제에 주로 사용되는 cross entropy loss르 사용한다.

In [19]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [20]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.4536, grad_fn=<NegBackward0>) tensor(2.3026)


uniform random digit으로 초기화 했으므로 정답 확률 갑슨 1/10에 수렵할 것을 예측할 수 있다.

`.backward`를 이용하여 미분을 계산한다.


In [21]:
loss = loss_func(outs, yb)

loss.backward()

`.backward`를 수행하게 되면 `requires_grad=True`인 `tensor`객체의 `.grad`에 미분 값이 저장된다.

In [22]:
bias.grad

tensor([-0.0168, -0.0140,  0.0242, -0.0543, -0.0163,  0.0904, -0.0030,  0.0456,
        -0.0026, -0.0533])

## 훈련 루프 작성 및 실행

네트워크를 학습시키기 위한 필요사항들을 모두 준비했다.     
- data
- parameter(model, weight, bias)를 가지고 있는 architecture
- loss_function     
이제 `for` loop, indexing, 함수 호출을 이용하여 학습을 진행할 수 있다.

In [23]:
lr = 0.5 
epochs = 2

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]
        # print(yb.shape)
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        
        with torch.no_grad(): # 이 계산 동안은 gradient를 추적하지 않는다.
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            weights.grad.zero_()
            bias.grad.zero_()


In [24]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0802, grad_fn=<NegBackward0>) tensor(1.)


In [25]:
# re-execute this cell for more samples
idx = random.randint(0, len(x_train))
example = x_train[idx:idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(8)


# torch.nn을 이용한 리펙토링

학습까지의 과정이 작동은 하지만 매우 번거롭다.      
`torch.nn`, `torch.optim`, `torch.utils.data`.

# Using `torch.nn.functional` for stateless computation

torch를 이용하여 기본적인 수식을 작성하기 전에 항상 해당 기능을 제공하는 함수라 pytorch 라이브러리에 있는지부터 확인하라

In [26]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [27]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0802, grad_fn=<NllLossBackward0>) tensor(1.)


# `torch.nn.Parameter`를 통해 상태(state)가 존재하는 함수를 정의하기 위해서는 `torch.nn`을 사용한다.

neural network는 최적화 과정을 거쳐야하므로 내부에 state가 존재한다(weight, bias). 우리가 추적하기를 원하는 모든 텐서들은 nn.Parameter로 정의되고 이는 model의 attribute에 저장되어 있다.

In [28]:
from torch import nn

class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

`.forward` 를 사용하면 파이썬의 `__call__` 매서드와 같이 nn.Module를 상속한 객체를 함수처럼 호출하였을 때 작동할 결과를 정의할 수 있다.     
`.forward`에는 `nn.Parameter`를 이용하여 정의한 state들을 이용하여 수행하는 연산을 정의한다.

In [29]:
def forward(self, xb: torch.Tensor) -> torch.Tensor:
    return xb @ self.weights + self.bias

MNISTLogistic.forward = forward

model = MNISTLogistic()
print(model(xb)[:4])
loss = loss_func(model(xb), yb)
loss.backward()
print(model.weights.grad[::17, ::2])

tensor([[ 0.2465, -0.8651, -0.2091, -0.2762, -0.1541,  0.5474,  0.0163,  0.1913,
         -0.0381,  0.5062],
        [ 0.0402, -0.6032, -0.0817, -0.1863, -0.1107,  0.6191,  0.9044,  0.2905,
          0.3296,  0.1488],
        [ 0.0451, -0.5915, -0.4251, -0.2839, -0.0162,  0.4792,  0.3028,  0.2725,
         -0.3696, -0.1772],
        [ 0.3880, -0.6863, -0.2375, -0.3959,  0.1112,  0.5058,  0.3352,  0.5348,
         -0.1852,  0.4321]], grad_fn=<SliceBackward0>)
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 6.3487e-03,  6.7917e-03,  6.5589e-03,  2.1063e-02,  7.6527e-03],
        [ 0.

`.Parameters` 매서드를 이용하면 우리 모델에 존재하는 모든 `torch.nn.Parameters`에 접근할 수 있다.

In [30]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0263,  0.0418, -0.0333,  ..., -0.0458, -0.0098,  0.0689],
        [ 0.0106, -0.0493,  0.0020,  ...,  0.0403,  0.0024, -0.0343],
        [-0.0546, -0.0131, -0.0484,  ...,  0.0060, -0.0103,  0.0097],
        ...,
        [ 0.0462, -0.0803,  0.0055,  ..., -0.0489,  0.0187, -0.0293],
        [ 0.0174,  0.0316,  0.0307,  ..., -0.0371,  0.0313, -0.0229],
        [-0.0496, -0.0374,  0.0703,  ...,  0.0433,  0.0019, -0.0422]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


즉 사용자는 더 이상 model 파라미터의 이름을 알고 있을 필요가 없다.     
따라서 아래와 같이 다른 모델에 대해 같은 train loop를 작성할 수 있다.

In [31]:
def fit():
    for epoch in range(epochs):
        for ii in range((n - 1) // bs + 1):
            start_idx = ii * bs
            end_idx = start_idx + bs
            xb = x_train[start_idx: end_idx]
            yb = y_train[start_idx: end_idx]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()
fit()

In [32]:
print(accuracy(model(xb), yb))

tensor(1.)


# Refactoring intermediate `torch.nn` components: network layers, optimizers, data handling

## Using torch.nn.Linear for the model definition

`nn.Module`을 상속하여 custom layer를 만드는 것보다 사전에 정의된 layer를 사용할 수 있다.

In [33]:
import textwrap

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")

torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [34]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)
    
    def forward(self, xb):
        return self.lin(xb)

In [35]:
model = MNISTLogistic()
print(loss_func(model(xb), yb))

tensor(2.3718, grad_fn=<NllLossBackward0>)


`nn.Linear`을 `model`의 child로 볼 수 있다.

In [36]:
print(*list(model.parameters()), sep='\n')

Parameter containing:
tensor([[-0.0299,  0.0109, -0.0134,  ...,  0.0154, -0.0264, -0.0009],
        [-0.0298,  0.0096, -0.0137,  ..., -0.0286, -0.0159,  0.0080],
        [-0.0097, -0.0036,  0.0186,  ..., -0.0036, -0.0074, -0.0109],
        ...,
        [ 0.0347, -0.0123, -0.0231,  ..., -0.0274, -0.0202,  0.0233],
        [ 0.0308, -0.0211, -0.0285,  ...,  0.0121, -0.0039,  0.0341],
        [-0.0223,  0.0314,  0.0147,  ...,  0.0260,  0.0226, -0.0201]],
       requires_grad=True)
Parameter containing:
tensor([-0.0202, -0.0227,  0.0329, -0.0169,  0.0061,  0.0007,  0.0225, -0.0029,
         0.0167, -0.0034], requires_grad=True)


# Applying gradients with `torch.optim.Optimizer`

## `torch.optim.Optimizer` 에 모델의 parameter들을 가르켜주면 최적화 과정을 손쉽게 해결할 수 있다.

In [37]:
from torch import optim

def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

In [38]:
model = MNISTLogistic()
opt = configure_optimizer(model)

print("before training:", loss_func(model(xb), yb), sep='\n\t')

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs
        xb = x_train[start_idx: end_idx]
        yb = y_train[start_idx: end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)
        opt.step()
        opt.zero_grad()

print("after training", loss_func(model(xb), yb), sep='\n\t')

before training:
	tensor(2.2517, grad_fn=<NllLossBackward0>)
after training
	tensor(2.2517, grad_fn=<NllLossBackward0>)


## organizing data with `torch.utils.data.Dataset`

`torch.utils.data.Dataset`을 이용하여 multiple data source에서 데이터 세트를 구성하고 indexing 할 수 있다.     
또한 input과 output의 transform을 효율적으로 수행할 수 있다.

In [39]:
from text_recognizer.data.util import BaseDataset

train_ds = BaseDataset(x_train, y_train)

In [40]:
BaseDataset??

In [41]:
model = MNISTLogistic()
opt = configure_optimizer(model)


for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        xb, yb = train_ds[ii * bs: ii * bs + bs]  # xb and yb in one line!
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.8349, grad_fn=<NllLossBackward0>)


## Batching up data with `torch.utils.data.DataLoader`

`DataLoader`를 이용하면 batch를 손쉽게 나눌 수 있다.     
`DataLoader`의 다른 옵션(num_workers, pin_memory)을 사용하면 training loop의 성능을 끌어올릴 수 있다.

In [42]:
from torch.utils.data import DataLoader

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [43]:
def fit(self: nn.Module, train_dataloader: DataLoader):
    opt = configure_optimizer(self)

    for epoch in range(epochs):
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

MNISTLogistic.fit = fit

In [44]:
model = MNISTLogistic()

model.fit(train_dataloader)

print(loss_func(model(xb), yb))

tensor(0.8498, grad_fn=<NllLossBackward0>)


아래의 이전 코드와 비교하면 매우 깔끔해진 것을 확인할 수 있다.

```python
lr = 0.5  # learning rate
epochs = 2  # how many epochs to train for

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()
```

## Swapping in another model

`.fit`의 성능을 체감하기 위해 다른 모델로 전환해본다.

In [45]:
from text_recognizer.models.mlp import MLP

MLP.fit = fit

`MLP`의 `.forward` 매서드를 확인하면 [`nn.Dropout`](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)
와 [`F.relu`](https://pytorch.org/docs/stable/generated/torch.nn.functional.relu.html) 같이 본 적이 없는 모듈과 함수가 포함되어 있는 것을 확인할 수 있으나 그 외에는 기존에 작성한 training loop와 잘 맞는 것을 확인할 수 있다.(`MLP`는 호출 가능하며, `X`를 input으로 받고 `y`를 예측한다.)

In [46]:
MLP.forward??

__init__을 살펴보면, nn.Module이 초기화되는 것을 확인할 수 있다.

In [47]:
MLP.__init__??

또한 MLP 모듈이 `data_config`를 요구하는 것을 확인할 수 있고, 선택적으로 `args`를 받는 것을 확인할 수 있다.      
지금은 최소한의 조건만 추가한다.

In [48]:
digits_to_9 = list(range(10))
data_config = {"input_dims": (784,), "mapping": {digit: str(digit) for digit in digits_to_9}}
data_config

{'input_dims': (784,),
 'mapping': {0: '0',
  1: '1',
  2: '2',
  3: '3',
  4: '4',
  5: '5',
  6: '6',
  7: '7',
  8: '8',
  9: '9'}}

In [49]:
model = MLP(data_config)
model

MLP(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

완성된 `MLP`는 우리의 `MNISTLogistic` 모델보다 조금 큰 것을 확인할 수 있다.

In [50]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0236,  0.0339, -0.0172,  ..., -0.0095,  0.0094, -0.0049],
        [-0.0068,  0.0085, -0.0114,  ..., -0.0100,  0.0260,  0.0141],
        [-0.0156, -0.0219,  0.0002,  ...,  0.0009,  0.0012,  0.0303],
        ...,
        [-0.0021, -0.0180,  0.0218,  ..., -0.0039, -0.0003, -0.0201],
        [ 0.0145,  0.0257,  0.0151,  ..., -0.0206,  0.0332,  0.0131],
        [-0.0131,  0.0047,  0.0017,  ...,  0.0253, -0.0139,  0.0013]],
       requires_grad=True)

하지만 시간이 좀 더 오래걸릴 뿐 훈련단계의 변화는 없다.

In [51]:
%%time

print("before training", loss_func(model(xb), yb))

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)
fit(model, train_dataloader)

print("after training", loss_func(model(xb), yb))

before training tensor(2.2893, grad_fn=<NllLossBackward0>)
after training tensor(0.1888, grad_fn=<NllLossBackward0>)
CPU times: user 27.5 s, sys: 350 ms, total: 27.9 s
Wall time: 28.4 s


# Extra goodies: data organization, validation, acceleration

CPU는 단일 연산에 특화되어 있기 때문에 DNN과 같은 병렬 행렬 연산이 필요한 작업에는 GPU를 사용하는 것이 작업을 훤씬 빠르게 끝낼 수 있다.

In [52]:
if torch.cuda.is_available():
    !nvidia-smi
else:
    print("☹️")

☹️


Pytorch는 CPU와 GPU를 모두 사용할 수 있고, 동시에도 사용가능하다. 따라서 `Tensor`가 CPU와 GPU 어디에 할당되어 있는지 설정해줄 필요가 있다.    
사용가능한 기기는 `torch.device`를 이용하여 확인할 수 있다.     
아래의 명령어는 모델의 파라미터를들 GPU로 이동시켜준다.(`.to`) 이는 `torch.nn.Parameter`를 사용하는 또다른 이점이다.      

In [53]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

loss_func(model(xb.to(device)), yb.to(device))

tensor(0.2234, grad_fn=<NllLossBackward0>)

BaseDataset의 또다른 기능으로, input에 사용할 변환을 `transform` 인자에, 정답 레이블에 적용할 변환을 `target_transform`에 대입할 수 있다.


In [54]:
def push_to_device(tensor):
    return tensor.to(device)

train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
train_dataloader = DataLoader(train_ds, batch_size=bs)

별도로 training code를 바꿔주어야 하는 것은 없다.

In [55]:
%%time

model = MLP(data_config)
model.to(device)

model.fit(train_dataloader)
print(loss_func(model(push_to_device(xb)), push_to_device(yb)))

tensor(0.3628, grad_fn=<NllLossBackward0>)
CPU times: user 27.5 s, sys: 537 ms, total: 28 s
Wall time: 29 s


Writing high performance GPU-accelerated neural network code is challenging.
There are many sharp edges, so the default
strategy is imitation (basing all work on existing verified quality code)
and conservatism bordering on paranoia about change.
For a casual introduction to some of the core principles, see
[Horace He's blogpost](https://horace.io/brrr_intro.html).

# Adding validation data and organizing data code with a `DataModule`

모델이 이미 확인한 데이터(training set)을 맞추는 것은 인상적이지 않다.     
모델 학습에 사용되지 않은 데이터(validation set)에 대해서 성능을 확인해본다.

일반적으로 데이터 로드 코드는 복잡하고, 동기화가 되기 힘들게 짜질 수 있다.     
적절한 `DataModule`코드를 작성하면 유용하게 쓰일 수 있다.

In [56]:
class MNISTDataModule:
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    def __init__(self, dir, bs=32):
        self.dir = dir
        self.bs = bs
        self.path = self.dir / self.filename

    def prepare_data(self):
        if not (self.path).exists():
            content = requests.get(self.url + self.filename).content
            self.path.open('wb').write(content)
    
    def setup(self):
        with gzip.open(self.path, 'rb') as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
        
        x_train, y_train, x_valid, y_valid = map(torch.tensor, (x_train, y_train, x_valid, y_valid))

        self.train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
        self.valid_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds, batch_size=self.bs, shuffle=True)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.valid_ds, batch_size=self.bs, shuffle=False) # valid는 shuffle False

`DataModule`에 대해서는 나중에 자세히 살펴본다.    
이제 `DataModule`을 fitting pipeline에 추가한다.


In [64]:
def fit(self: nn.Module, datamodule):
    datamodule.prepare_data()
    datamodule.setup()

    val_dataloader = datamodule.val_dataloader()

    self.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)

    print("before start of training:", valid_loss / len(val_dataloader))

    opt = configure_optimizer(self)
    train_dataloader = datamodule.train_dataloader()
    for epoch in range(epochs):
        self.train()
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

        self.eval()
        with torch.no_grad():
            valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)
        
        print(epoch, valid_loss / len(val_dataloader))
    
MNISTLogistic.fit = fit
MLP.fit = fit

In [65]:
model = MLP(data_config)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=32)

model.fit(datamodule=datamodule)

before start of training: tensor(2.3026)
0 tensor(0.1813)
1 tensor(0.1101)


이제 아래의 코드만드로 학습이 가능하다.

In [66]:
model = MLP(data_config)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=32)

model.fit(datamodule=datamodule)

before start of training: tensor(2.3054)
0 tensor(0.1719)
1 tensor(0.1072)


`.fit` 매서드에서 주의깊게 봐야할 점은 아래와 같다.     
- `self.eval` vs `self.train`: [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)
and
[BatchNorm](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) 과 같이 훈련과 검증 시 다르게 동작하는 레이어들의 training과 evaliadation 단계의 동작을 손쉽게 바꿔준다.      
- `torch.no_grad` 를 사용하면 tensor계산시에 parameter들을 학습시키 위해 사용되는 gradient를 구하기 위한 tracking을 적용하지 않는다.       
검증시와 같이 모델 훈련에 사용되지 않는 과정의 경우 필요하다.

# Excercise

## Try out different hyperparameters for the `MLP` and for training

mlp를 보면 dropout 옵션을 설정할 수 있는 것을 알 수 있다.

In [138]:
MLP.__init__??

As the type signature indicates, `args` is an `argparse.Namespace`.
[`argparse` is used to build command line interfaces in Python](https://realpython.com/command-line-interfaces-python-argparse/),
and later on we'll see how to configure models
and launch training jobs from the command line
in the FSDL codebase.

For now, we'll do it by hand, by passing a dictionary to `Namespace`.

Edit the cell below to change the `args`, `epochs`, and `b`atch `s`ize.

Can you get a final `valid`ation `acc`uracy of 98%?
Can you get to 95% 2x faster than the baseline `MLP`?

In [111]:
import argparse
from argparse import Namespace

parser = argparse.ArgumentParser()
parser.add_argument('--fc_dropout')

args = parser.parse_args(['--fc_dropout=0.2'])
print(args.fc_dropout)
args = vars(args) if parser is not None else {}
# 
print(args)

0.2
{'fc_dropout': '0.2'}


In [139]:
model = MLP(data_config, args=args)

In [140]:
model.__dict__

{'_backward_hooks': OrderedDict(),
 '_buffers': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_load_state_dict_post_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('fc1',
               Linear(in_features=784, out_features=1024, bias=True)),
              ('dropout', Dropout(p=0.2, inplace=False)),
              ('fc2', Linear(in_features=1024, out_features=128, bias=True)),
              ('fc3', Linear(in_features=128, out_features=10, bias=True))]),
 '_non_persistent_buffers_set': set(),
 '_parameters': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 'args': {'fc_dropout': 0.2},
 'data_config': {'input_dims': (784,),
  'mapping': {0: '0',
   1: '1',
   2: '2',
   3: '3',
   4: '4',
   5: '5',
   6: '6',
   7: '7',
   8: '8',
   9: '9'}},
 'training': True}

In [114]:
%%time
import argparse
from argparse import Namespace

parser = argparse.ArgumentParser()
parser.add_argument('--fc_dropout', action='store', type=float)
args = parser.parse_args(['--fc_dropout', '0.2'])
print(args.fc_dropout)
epochs = 2
bs = 32


def configure_optimzier(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), le=3e-4)


model = MLP(data_config, args=args)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=bs)

model.fit(datamodule=datamodule)

0.2
before start of training: tensor(2.2991)
0 tensor(0.1426)
1 tensor(0.0852)
CPU times: user 58.5 s, sys: 3.71 s, total: 1min 2s
Wall time: 1min 5s


In [93]:
val_dataloader = datamodule.val_dataloader()
valid_acc = sum(accuracy(model(xb), yb) for xb, yb in val_dataloader) / len(val_dataloader)
valid_acc

tensor(0.9675)

# Write your own `nn.Module`

Designing new models is one of the most fun
aspects of building an ML-powered application.

Can you make an `nn.Module` that looks different from
the standard `MLP` but still gets 98% validation accuracy or higher?
You might start from the `MLP` and
[add more layers to it](https://i.imgur.com/qtlP5LI.png)
while adding more bells and whistles.
Take care to keep the shapes of the `Tensor`s aligned as you go.

Here's some tricks you can try that are especially helpful with deeper networks:
- Add [`BatchNorm`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html)
layers, which can improve
[training stability and loss conditioning](https://myrtle.ai/how-to-train-your-resnet-7-batch-norm/)
- Add a linear "skip connection" layer that is applied to the inputs and whose outputs are added directly to the last layer's outputs
- Use other [activation functions](https://pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions),
like [selu](https://pytorch.org/docs/stable/generated/torch.nn.functional.selu.html)
or [mish](https://pytorch.org/docs/stable/generated/torch.nn.functional.mish.html)

If you want to make an `nn.Module` that can have different depths,
check out the
[`nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) class.

In [127]:
torch.zeros(3, 2)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [143]:
import numpy as np
from typing import Dict, Any

FC1_DIM = 1024
FC2_DIM = 1024
FC_DROPOUT=0.2

class YourModel(nn.Module):
    
    def __init__(self, data_config: Dict[str, Any], args: argparse.Namespace = None) -> None:
        super().__init__()
        self.args = vars(args) if args is not None else {}
        self.data_config = data_config

        input_dim = np.prod(self.data_config["input_dims"])
        num_classes = len(self.data_config["mapping"])

        fc1_dim = self.args.get("fc1", FC1_DIM)
        fc2_dim = self.args.get("fc2", FC2_DIM)
        dropout_p = self.args.get("fc_dropout", FC_DROPOUT)

        self.ps = nn.Parameter(torch.zeros(input_dim, 10))
        self.fc1 = nn.Linear(input_dim, fc1_dim)
        self.act1 = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm1d(fc1_dim)
        self.dropout1 = nn.Dropout(dropout_p)
        
        self.fc2 = nn.Linear(fc1_dim, fc2_dim)
        self.act2 = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm1d(fc2_dim)
        self.dropout2 = nn.Dropout(dropout_p)
        
        self.fc3 = nn.Linear(fc2_dim, num_classes)

    def forward(self, xb):
        x_skip = xb @ self.ps
        xb = self.dropout1(self.act1(self.batch_norm1(self.fc1(xb))))
        xb = self.dropout2(self.act2(self.batch_norm2(self.fc2(xb))))
        xb = self.fc3(xb)

        return xb + x_skip

YourModel.fit = fit  # don't forget this!

In [144]:
model = YourModel(data_config)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=bs)

model.fit(datamodule=datamodule)

before start of training: tensor(2.3082)
0 tensor(0.0766)
1 tensor(0.0445)


In [145]:
val_dataloader = datamodule.val_dataloader()
valid_acc = sum(accuracy(model(xb), yb) for xb, yb in val_dataloader) / len(val_dataloader)
valid_acc

tensor(0.9871)